In [1]:
%matplotlib inline

In [2]:
import os
import asyncio
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as mdt
import MDAnalysis as mda
import arcd
import arcd.distributed as arcdd

Tensorflow/Keras not available


In [3]:
# setup logging
# executing this file sets the variable LOGCONFIG, which is a dictionary of logging presets 
%run ../resources/logconf.py

In [4]:
LOGCONFIG

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standardFormatter': {'class': 'logging.Formatter',
   'format': '(%(levelname)s)%(name)s: %(message)s'},
  'msgOnly': {'class': 'logging.Formatter', 'format': '%(message)s'}},
 'handlers': {'stdout': {'class': 'logging.StreamHandler',
   'level': 'NOTSET',
   'formatter': 'msgOnly'},
  'warnout': {'class': 'logging.StreamHandler',
   'level': 'WARN',
   'formatter': 'standardFormatter'},
  'initf': {'class': 'logging.FileHandler',
   'level': 'INFO',
   'mode': 'w',
   'filename': 'initialization.log',
   'formatter': 'standardFormatter'},
  'stdf': {'class': 'logging.FileHandler',
   'level': 'INFO',
   'mode': 'w',
   'filename': 'simulation.log',
   'formatter': 'standardFormatter'}},
 'loggers': {'': {'level': 'INFO', 'handlers': ['stdf', 'warnout']},
  'openpathsampling.initialization': {'level': 'INFO',
   'handlers': ['initf'],
   'qualname': 'openpathsampling.initialization',
   'propagate': 0}}}

In [5]:
# setup working directory
scratch_dir = "/home/think/scratch/arcd_distributed/"

workdir = os.path.join(scratch_dir, "Committors_test")

if not os.path.isdir(workdir):
    os.mkdir(workdir)

In [6]:
# you can either modify single values or use it as is to get the same setupt as in the OPS default logging config file
# you could e.g. do LOGCONF['handlers']['stdf']['filename'] = new_name to change the filename of the log
# the default is to create 'simulation.log' and 'initialization.log' in the current working directory
import logging.config
LOGCONFIG["handlers"]["stdf"]["filename"] = os.path.join(workdir, "simulation_committors.log")
LOGCONFIG["handlers"]["initf"]["filename"] = os.path.join(workdir, "initlog_committors.log")
logging.config.dictConfig(LOGCONFIG)

# now the actual setup

In [7]:
# import state functions
import state_funcs

In [8]:
# this is just to have a look at the file content
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
import IPython

with open('state_funcs.py') as f:
    code = f.read()

formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
    formatter.get_style_defs('.highlight'),
    highlight(code, PythonLexer(), formatter)))

In [9]:
wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(state_funcs.alpha_R)

wrapped_C7_eq = arcdd.TrajectoryFunctionWrapper(state_funcs.C7_eq)

In [10]:
init_confs = [(arcdd.Trajectory(os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"),
                                os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro")
                                ), 0) for _ in range(4)]
# engine_kwargs
gro = os.path.join(scratch_dir, "gmx_infiles/conf.gro")
top = os.path.join(scratch_dir, "gmx_infiles/topol.top")
mdp = arcdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))

engine_kwargs = {"gro_file":gro, "top_file":top, "mdrun_extra_args":"-nt 1"}

simulation = arcdd.CommittorSimulation(workdir=workdir,
                                       starting_configurations=init_confs,
                                       states=[wrapped_C7_eq, wrapped_alphaR],
                                       engine_cls=arcdd.GmxEngine,
                                       engine_kwargs=engine_kwargs,
                                       engine_mdconfig=mdp,
                                       T=mdp["ref-t"][0],
                                       walltime_per_part=0.01,
                                       n_max_concurrent=5,
                                       two_way=True,
                                       wait_time_on_crash=5,
                                       max_retries_on_crash=5,
                                       )

In [17]:
ret = await simulation.run(100)

/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` i

In [18]:
print(simulation.max_retries_on_crash)
print(simulation.wait_time_on_crash)

5
5


In [19]:
ret

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]])

In [20]:
simulation.states_reached

array([[192.,   8.],
       [195.,   5.],
       [192.,   8.],
       [196.,   4.]])

In [21]:
simulation.states_reached_per_shot

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]])